# Анализ вакансий DS, DA в телеграм каналах

#  План работы над проектом

**Цель проекта**:
* Составить портрет сотрудника, которого чаще всего готовы принять на позиции DS, DA, а также оценить условия труда, предлогаемые в этих вакансиях. На основе анализа можно будет проследить как меняется рынок труда, выделить тенденции. Это поможет соискателям лучше понимать нанимающие компании, оценить и сопоставить свои ожидания и предложения компаний.

**При описании портрета сотрудника и оценке условий труда, будем учитывать следующее:**
* образование
* нанимающую компанию
* размер компании
* опыт сотрудника (грейд)
* требуемый стек технологий
* формат работы (удаленный/офис)
* зарплата

**В ходе исследования проверим следующие гипотезы:**
1. Больше всего вакансий, предлагающих работу в офисе, чем удаленный формат.
2. Приблизиттельное одинаковое количество вакансий для DA и DS
3. Больше вакансий от крупных компаний, чем от стартапов. 
4. Больше всего вакансий для специалистов с опытом, чем для junior специалистов.
5. В зимнее месяцы количество вакансий больше, чем в летние месяцы. 




# Этапы выполнения проекта.
1. Парсер телеграм-каналов с вакансиями DS, DA.
        * выбрать необходимые телеграм-каналы
        * выбрать период за который будут анализироваться вакансии
        * установить лимит выгружаемых вакансий
        * сформировать csv-файл с первоначальными данными о вакансиях
2. Предобработка данных:
        * Очистить и подготовить текст для дальнейшей работы (с использованием регулярных выражений, лемматизация, библиотеки    ntlk с стоп-словами).
        * Исследовать пропущенные значения
        * Проверить наличие дубликатов
        * Работа с типами данных
        * Создание необходимых колонок
3. Исследовательский анализ данных:
        * Обнаружение аномалий, выбросов, интересных фактов и зависимостей.
        * Поиск ответов на выдвинутые гипотезы
        * Визуальзация данных с 
4. Оформление результатов исследования в понятный и функциональный дашборд. 
        * визуализировать ответы на выдвинутые гипотезы
        * визуализировать потррет типичного специалиста DS и DA

Для анализа вакансий будем использовать следующие чаты.

* https://t.me/foranalysts
* https://t.me/analyst_job
* https://t.me/biheadhunter
* https://t.me/datasciencejobs
* https://t.me/bds_job
* https://t.me/datajobschannel
* https://t.me/data_hr

# Парсер телеграм-каналов

In [1]:
# !pip3 install pyrogram tgcrypto

In [2]:
# !pip3 install python-dotenv

In [3]:
import pyrogram
import requests
import numpy as np
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import os
from pyrogram import Client
import pandas as pd
import json
from datetime import datetime
import re
from IPython.core.display import display, HTML, clear_output
import ipywidgets as widgets
from urllib.parse import urlparse

/var/folders/vf/z1gk_tyn2zsf73w6bspmxgjm0000gn/T/ipykernel_79367/1215047102.py:12: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML, clear_output
/var/folders/vf/z1gk_tyn2zsf73w6bspmxgjm0000gn/T/ipykernel_79367/1215047102.py:12: DeprecationWarning: Importing clear_output from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML, clear_output


In [4]:
pyrogram.__version__

'2.0.51'

In [5]:
dotenv_path = os.path.join('dot.env')
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path)

In [6]:
API_ID = os.environ['API_ID']
API_HASH = os.environ['API_HASH']

In [7]:
# выгружаем данные после работы скрипта
# %%bash
# python3 pyro_run.py

In [8]:
# читаем и сохраняем данные в переменную
telegram = pd.read_csv('telegram.csv')

In [64]:
telegram.head(3)

,chat,message_id,date,text,entities,chat_id,title,url,host
0,"{\n ""_"": ""Chat"",\n ""id"": -1001137236002,...",4730,2022-09-09 17:05:10,Product Analyst\nв Safeguard Global — поставщи...,"[\n {\n ""_"": ""MessageEntity"",\n ...",-1001137236002,Job for Analysts & Data Scientists,https://www.linkedin.com/jobs/view/3243717640/,www.linkedin.com
1,"{\n ""_"": ""Chat"",\n ""id"": -1001137236002,...",4729,2022-09-09 15:12:08,Бизнес-аналитик/Менеджер проекта\nв Simble — I...,"[\n {\n ""_"": ""MessageEntity"",\n ...",-1001137236002,Job for Analysts & Data Scientists,https://geekjob.ru/hZ3N,geekjob.ru
2,"{\n ""_"": ""Chat"",\n ""id"": -1001137236002,...",4728,2022-09-09 09:00:11,Lead Data Engineer\nв Cardsmobile — компания р...,"[\n {\n ""_"": ""MessageEntity"",\n ...",-1001137236002,Job for Analysts & Data Scientists,https://geekjob.ru/hZ3K,geekjob.ru


In [10]:
telegram.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8046 entries, 0 to 8045
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   chat        5048 non-null   object
 1   message_id  8046 non-null   int64 
 2   date        8046 non-null   object
 3   text        7512 non-null   object
 4   entities    7345 non-null   object
dtypes: int64(1), object(4)
memory usage: 314.4+ KB


In [11]:
# избавляемся от пропусков
telegram = telegram.dropna().reset_index(drop=True) 

In [70]:
telegram.shape

(4580, 9)

В отдельную колонку сохраним информацию о id чата и его названии.

In [12]:
telegram["chat_id"] = telegram["chat"].apply(lambda x: json.loads(x)['id'])
telegram["title"] = telegram["chat"].apply(lambda x: json.loads(x)['title'])

In [68]:
# В колонке data изменит тип данных
telegram.date = pd.to_datetime(telegram['date'])

In [62]:
telegram.date.sort_values(ascending=False)

0      2022-09-09 17:05:10
1      2022-09-09 15:12:08
847    2022-09-09 14:39:26
2850   2022-09-09 13:46:35
1324   2022-09-09 10:02:44
               ...        
4320   2019-02-18 13:24:38
4321   2019-02-10 12:12:21
4322   2019-02-06 14:14:56
4323   2019-01-31 22:29:30
4324   2019-01-29 20:02:49
Name: date, Length: 4580, dtype: datetime64[ns]

In [66]:
telegram.head(1)

,chat,message_id,date,text,entities,chat_id,title,url,host
0,"{\n ""_"": ""Chat"",\n ""id"": -1001137236002,...",4730,2022-09-09 17:05:10,Product Analyst\nв Safeguard Global — поставщи...,"[\n {\n ""_"": ""MessageEntity"",\n ...",-1001137236002,Job for Analysts & Data Scientists,https://www.linkedin.com/jobs/view/3243717640/,www.linkedin.com


Кнопка для просмотра выгруженных вакансий.

In [17]:
button = widgets.Button(description="Показать")
output = widgets.Output()

display(button, output)

counter = 0
def on_clicked(b):
    with output:
        try:
            global counter
            clear_output()
            print('№ вакансии: ', counter)
            print(telegram['text'][counter])
            counter += 1
        except Exception as e:
            print("Вакансии закончились")
            
button.on_click(on_clicked)

Button(description='Показать', style=ButtonStyle())

Output()

## Вывод
* Данные о вакансиях парсили из 7 телеграм-каналов
* Всего 4580 строк
* Данные выгруженны за период с 29.01.2019 по 09.09.2022

# Извлечение информации по ссылкам в вакансиях

## Выгрузим ссылки в отдельную колонку датасета - `url`

In [14]:
def get_links(x, regexp='http\S+'): ###
    try:
        for i in json.loads(x['entities']):
            if i['type'] == 'MessageEntityType.TEXT_LINK':
                return i['url']
            elif i['type'] == 'MessageEntityType.URL':
                url = re.findall(regexp, x['text'])[0]
                return url
    except:
        return None

In [15]:
telegram['url'] = telegram.apply(get_links, axis=1)

In [16]:
telegram['url'][0]

'https://www.linkedin.com/jobs/view/3243717640/'

## Найдем наиболее встречающиеся сайты, с которых поступает информация о вакансиях

In [26]:
# функция для поиска hostname url ссылки
def hostname(link):
    o = urlparse(link)
    host = o.hostname
    return host

In [48]:
telegram['host'] = telegram['url'].apply(lambda x: hostname(x))

Выведем 30 hostname, ссылка на которые встречаются больше всего. Будем вытаскивать информацию из ссылки, если она встречается больше 5 раз и ведет на сайт с вакансиями. 

In [43]:
telegram.groupby('host')['text'].agg('count').sort_values(ascending=False).head(30)

host
geekjob.ru                                   733
ya.cc                                        203
hh.ru                                        131
t.me                                         115
purninja-st.s3.eu-central-1.amazonaws.com     40
is.gd                                         31
otus.ru                                       26
i.ibb.co                                      26
spb.hh.ru                                     24
practicum.yandex.ru                           23
www.notion.so                                 23
telegra.ph                                    14
proglib.io                                    14
clck.ru                                       14
www.youtube.com                               12
ctrl2go.solutions                             12
praktikum.yandex.ru                           12
www.linkedin.com                              10
netology.ru                                    9
yandex.ru                                      9
your.gms.tech  

## Напишем функции для парсинга ссылок

In [18]:
def parse_site(url, element='div', extra_info=None):
    try:
        data = requests.get(url).text 
        soup = BeautifulSoup(data)
        elements = []
        for extra in extra_info:
            elements += soup.find_all(element, extra)
        html = ''
        for i in elements:
            html += str(i)
        return html
    except Exception as e:
        return '<p>Не удалось получить элемент</p>'

In [54]:
# HTML(parse_site(telegram['url'][1], element='section', extra_info=[{'class': 'col s12 m12 main'}]))

In [45]:
# def geekjob(url):
#     response = requests.get(url)
#     if response.status_code > 305:
#         soup = BeautifulSoup(response.text)
        
#         title = soup.find_all('h1')
#         company = soup.find_alll('<h5 class="company-name')

In [52]:
# geekjob(telegram['url'][2])

In [20]:
# extra_infos = [{'class': 'col s12 m12 main'},# geekjob
#                {'class': 'p5'} #
               
#                {'class': 'bp-Vacancy__description bp-VacancyDescription'}, 
#                {'class': 'vacancy-description'},
#                {'class': 'section__block panel js-show'}]

# element = [{'element': 'section'},
#           {'element': 'div'}]

# ###
# telegram['parsed_text'] = telegram['url'].apply(lambda x: parse_site(x, extra_info=extra_infos))

KeyboardInterrupt: 

In [24]:
telegram['url'].unique()[0:10]

array(['https://www.linkedin.com/jobs/view/3243717640/',
       'https://geekjob.ru/hZ3N', 'https://geekjob.ru/hZ3K',
       'https://geekjob.ru/hZ3Z', 'https://geekjob.ru/hZ3M',
       'https://geekjob.ru/hZ2w',
       'https://www.linkedin.com/jobs/view/3242258629/',
       'https://geekjob.ru/hZ2f', 'https://geekjob.ru/hZ2a',
       'https://geekjob.ru/hZ2U'], dtype=object)